In [1]:
import pickle
with open('data/processed_data.pkl','rb') as f:
    data = pickle.load(f)

In [2]:
type(data)

tuple

In [8]:
weight_dict,id2idx = load_baseline()

In [12]:
from temp.models import *

AppRegistryNotReady: Apps aren't loaded yet.

In [7]:
import json

In [10]:
from django.conf import settings

settings.configure()

In [5]:
def load_baseline():
    with open('data/edge_query_dict.json', 'r') as f:
        edge_query_dict = json.load(f)
    with open('data/id2idx.json', 'r') as f:
        id2idx = json.load(f)
    return edge_query_dict, id2idx

def query_weight(id_i, id_j, weight: dict, id2idx: dict) -> list:
    idx_i, idx_j = id2idx[id_i], id2idx[id_j]
    key = str(idx_i) + '_' + str(idx_j)
    queried_results = weight[key]
    return queried_results

def sigmoid(x):
    return 1 / (1 + math.exp(-x))


def compute_score(cross_feature, n_citation):
    impact = sigmoid(n_citation / 10)
    score = cross_feature * impact
    return score

def get_value(id_i,id_j):
    try:
        ans = query_weight(id_i,id_j,weight_dict,id2idx)
        ans[2] = compute_score(ans[0],ans[1])
        return tuple(ans)
    except:
        ans = [0,node_dict[id_j]['citation'],0]
        ans[2] = compute_score(ans[0],ans[1])
        return tuple(ans)
    
def gen_graph(index):
    index = str(index)
    g = nx.DiGraph()
    root_name = node_dict[index]['title']
    g.add_node(index,label = root_name,showValue = 1.5,relevance = 1.0, impact = node_dict[index]['citation'], score = 1.0,modularity_class = 'self')
    nb_list = []
    co_list = []
    nb_set = set()
    nb_set.add(index)
    for nd in node_dict[index]['references']:
        nb_list.append((nd,get_value(index,nd),'references'))
        nb_set.add(nd)
    for nd in citation_dict.get(index,[]):
        nb_list.append((nd,get_value(index,nd),'cited'))
        nb_set.add(nd)
    for aut in node_dict[index]['authors']:
        for nd in author_dict[aut]:
            if nd not in nb_set:
                co_list.append((nd,get_value(index,nd),'co-author'))
                nb_set.add(nd)
    nb_list.sort(key = lambda x:x[1][2],reverse = True)
    nb_list = list(filter(lambda x:x[1][2]>0.0,nb_list))[:10]
    co_list.sort(key = lambda x:x[1][2],reverse = True)
    co_list = list(filter(lambda x:x[1][2]>0.0,co_list))[:15-len(nb_list)]
    if len(nb_list) > 0:
        val_max = nb_list[0][1][2]
        for nd,val,clas in nb_list:
            showValue,relevance,impact,score = 0.5*val[2]/val_max+0.7,val[0],val[1],val[2]
            g.add_node(nd,label = node_dict[nd]['title'],showValue = showValue,relevance = relevance, impact = impact, score = score, modularity_class = clas)
            g.add_edge(nd,index,weight = showValue)
    if len(co_list) > 0:
        val_max = co_list[0][1][2]
        for nd,val,clas in co_list:
            showValue,relevance,impact,score = 0.4*val[2]/val_max+0.3,val[0],val[1],val[2]
            g.add_node(nd,label = node_dict[nd]['title'],showValue = showValue,relevance = relevance, impact = impact, score = score, modularity_class = clas)
            g.add_edge(nd,index,weight = showValue)
    return g

In [2]:
node_dict,citation_dict,author_dict,kw_dic,title_dic = data

In [25]:
sum([len(_) for _ in kw_dic.values()])

5465652

In [26]:
len(kw_dic['data mining'])

55241

In [4]:
with open('data/keywords.pkl','wb') as f:
    pickle.dump((kw_dic,title_dic),f)

In [3]:
len(kw_dic)

58815

In [4]:
list(node_dict.keys())[0]

'100001334'

In [20]:
len(author_dict)

604712

In [21]:
from tqdm import tqdm

In [22]:
a = [x for x in tqdm(range(10))]

100%|███████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 55115.69it/s]


In [5]:
node_dict['100001334']

{'title': 'Ontologies in HYDRA - Middleware for Ambient Intelligent Devices.',
 'authors': ['Peter Kostelnik', 'Martin Sarnovsky', 'Jan Hreno'],
 'venue': 'AMIF',
 'year': 2009,
 'citation': 2,
 'keywords': ['Middleware',
  'Ambient intelligence',
  'Ontology (information science)',
  'Database',
  'Lernaean Hydra'],
 'abstract': '',
 'references': []}

In [14]:
from tqdm import tqdm

In [14]:
import networkx as nx

In [16]:
import math

In [17]:
g = gen_graph('100001334')

In [18]:
g.nodes

NodeView(('100001334', '1443096424'))

In [19]:
g.nodes['1443096424']

{'label': 'Evaluating Energy Profiles as Resource of Context and as Added Value in Integrated and Pervasive Socio-Medical Technologies using LinkSmart Middleware',
 'showValue': 1.2,
 'relevance': 0.329912438089707,
 'impact': 1,
 'score': 0.17319716368753046,
 'modularity_class': 'cited'}

In [28]:
max([int(x) for x in node_dict.keys()])

1879660786

In [26]:
max([len(';'.join(x['keywords'][:5])) for x in node_dict.values()])

236

In [2]:
import pickle
author_dict = pickle.load(open('data/citationv1_author.pkl','rb'))
node_dict = pickle.load(open('data/citationv1.pkl','rb'))

In [7]:
import networkx as nx

In [3]:
node_dict[0]

{'title': 'Automated Deduction in Geometry: 5th International Workshop, ADG 2004, Gainesville, FL, USA, September 16-18, 2004, Revised Papers (Lecture Notes in Computer ... / Lecture Notes in Artificial Intelligence)',
 'authers': ['Hoon Hong', 'Dongming Wang'],
 'year': 2006,
 'venue': '',
 'index': 0,
 'references': [],
 'abstract': '',
 'cited': []}

In [9]:
def gen_graph(index):
    g = nx.DiGraph()
    root_name = node_dict[index]['title']
    g.add_node(index,label = root_name,modularity_class = 'self')
    for nd in node_dict[index]['references']:
        g.add_node(nd,label = node_dict[nd]['title'],modularity_class = 'references')
        g.add_edge(nd,index)
    for nd in node_dict[index]['cited']:
        g.add_node(nd,label = node_dict[nd]['title'],modularity_class = 'cited')
        g.add_edge(index,nd)
    for aut in node_dict[index]['authers']:
        for nd in author_dict[aut]:
            if nd not in g.nodes:
                g.add_node(nd,label = node_dict[nd]['title'],modularity_class = 'co-author')
                g.add_edge(index,nd)
    return g

In [10]:
g = gen_graph(436405)

In [13]:
for node in g.nodes:
    print(g.nodes[node])

{'label': 'Information geometry of U-Boost and Bregman divergence', 'modularity_class': 'self'}
{'label': 'A decision-theoretic generalization of on-line learning and an application to boosting', 'modularity_class': 'references'}
{'label': 'Boosting as entropy projection', 'modularity_class': 'references'}
{'label': 'Information geometry of the EM and em algorithms for neural networks', 'modularity_class': 'references'}
{'label': 'Neural Networks for Pattern Recognition', 'modularity_class': 'references'}
{'label': 'Logistic Regression, AdaBoost and Bregman Distances', 'modularity_class': 'references'}
{'label': 'MadaBoost: A Modification of AdaBoost', 'modularity_class': 'references'}
{'label': 'Boosting a weak learning algorithm by majority', 'modularity_class': 'references'}
{'label': 'The Strength of Weak Learnability', 'modularity_class': 'references'}
{'label': 'The nature of statistical learning theory', 'modularity_class': 'references'}
{'label': 'Interpreting Kullback-Leibler 

In [14]:
a = '1'
str(a)

'1'

In [15]:
a = '"1"'
print(a)

"1"


In [1]:
import pickle
from rake_nltk import Rake

In [3]:
r = Rake()
nodes = pickle.load(open('data/citationv1.pkl','rb'))

In [9]:
nodes[2]

{'title': 'Performance engineering in industry: current practices and adoption challenges',
 'authers': ['Ahmed E. Hassan', 'Parminder Flora'],
 'year': 2007,
 'venue': 'Proceedings of the 6th international workshop on Software and performance',
 'index': 2,
 'references': [],
 'abstract': 'This panel session discusses performance engineering practices in industry. Presentations in the session will explore the use of lightweight techniques and approaches in order to permit the cost effective and rapid adoption of performance modeling research by large industrial software systems.',
 'cited': []}

In [10]:
r.extract_keywords_from_text(nodes[2]['abstract'])
r.get_ranked_phrases_with_scores()

[(32.0, 'panel session discusses performance engineering practices'),
 (16.0, 'large industrial software systems'),
 (10.5, 'performance modeling research'),
 (4.0, 'rapid adoption'),
 (4.0, 'lightweight techniques'),
 (4.0, 'cost effective'),
 (3.5, 'session'),
 (1.0, 'use'),
 (1.0, 'presentations'),
 (1.0, 'permit'),
 (1.0, 'order'),
 (1.0, 'industry'),
 (1.0, 'explore'),
 (1.0, 'approaches')]

In [11]:
import json

In [14]:
a
    nodes = f.readlines()

In [15]:
nodes[0]

'{"id": "100001334", "title": "Ontologies in HYDRA - Middleware for Ambient Intelligent Devices.", "authors": [{"name": "Peter Kostelnik", "id": "2702511795"}, {"name": "Martin Sarnovsky", "id": "2041014688"}, {"name": "Jan Hreno", "id": "2398560122"}], "venue": {"raw": "AMIF"}, "year": 2009, "n_citation": 2, "page_start": "43", "page_end": "46", "doc_type": "", "publisher": "", "volume": "", "issue": "", "fos": [{"name": "Lernaean Hydra", "w": 0.4178039}, {"name": "Database", "w": 0.4269269}, {"name": "World Wide Web", "w": 0.415332377}, {"name": "Ontology (information science)", "w": 0.459045082}, {"name": "Computer science", "w": 0.399807781}, {"name": "Middleware", "w": 0.5905041}, {"name": "Ambient intelligence", "w": 0.5440575}], "references": []}\n'

In [67]:
for aut in nds['136064334']['authors']:
    for nd in author_dict[aut]:
        print(nd)

136064334


In [66]:
nds['136064334']

{'title': 'Some comments on human archival memory as a very large data base',
 'authors': ['R. S. Nickerson'],
 'venue': 'very large data bases',
 'year': 1977,
 'citation': 2,
 'keywords': ['Data',
  'Information retrieval',
  'Data mining',
  'Database',
  'Computer science'],
 'abstract': 'Human archival memory is discussed as a very large data base. Attention is focused primarily on the questions: What is stored? and How is what is stored retrieved and used? The relevance of the study of human memory to the design of computer data bases is considered.',
 'references': []}

In [56]:
nds['100001334']

{'title': 'Ontologies in HYDRA - Middleware for Ambient Intelligent Devices.',
 'authors': ['Peter Kostelnik', 'Martin Sarnovsky', 'Jan Hreno'],
 'venue': 'AMIF',
 'year': 2009,
 'citation': 2,
 'keywords': ['Middleware',
  'Ambient intelligence',
  'Ontology (information science)',
  'Database',
  'Lernaean Hydra'],
 'abstract': '',
 'references': []}

In [16]:
nodes[1]

'{"id": "1000018889", "title": "Remote Policy Enforcement for Trusted Application Execution in Mobile Environments", "authors": [{"name": "Fabio Martinelli", "id": "2107438709", "org": "Istituto di Informatica e Telematica, Consiglio Nazionale delle Ricerche, Pisa, Italy#TAB#"}, {"name": "Ilaria Matteucci", "id": "2311169168", "org": "Istituto di Informatica e Telematica, Consiglio Nazionale delle Ricerche, Pisa, Italy#TAB#"}, {"name": "Andrea Saracino", "id": "2140477378", "org": "Dipartimento di Ingegneria dell\'Informazione, Universit di Pisa, Pisa, Italy and Istituto di Informatica e Telematica, Consiglio Nazionale delle Ricerche, Pisa, Italy#TAB#"}, {"name": "Daniele Sgandurra", "id": "252575161", "org": "Istituto di Informatica e Telematica, Consiglio Nazionale delle Ricerche, Pisa, Italy#TAB#"}], "venue": {"raw": "international conference on trusted systems", "id": "2760518661"}, "year": 2013, "n_citation": 2, "page_start": "70", "page_end": "84", "doc_type": "Conference", "publ

In [58]:
author_dict['Andrea Saracino']

['1000018889', '1597048264', '175930347']

In [61]:
kw_dic['data mining']

[('100014528', 0.451454639),
 ('1000228583', 0.456178755),
 ('1000250121', 0.456649959),
 ('100028214', 0.458733618),
 ('1000328014', 0.4454233),
 ('100034732', 0.454395),
 ('100036502', 0.473031968),
 ('100040114', 0.454733878),
 ('100040545', 0.418463737),
 ('100041910', 0.45997867),
 ('10004740', 0.4620256),
 ('100055053', 0.423765659),
 ('100059100', 0.446314067),
 ('100059397', 0.473002732),
 ('100069005', 0.454070926),
 ('1000699894', 0.4648623),
 ('1000727832', 0.458965272),
 ('100077324', 0.464521676),
 ('100082076', 0.455378741),
 ('100088125', 0.452119917),
 ('1000902583', 0.4145464),
 ('1000946100', 0.445888668),
 ('1000950318', 0.449208975),
 ('10012888', 0.453852355),
 ('10013091', 0.435298949),
 ('100133193', 0.456425369),
 ('100133723', 0.4660011),
 ('100138144', 0.450856328),
 ('1001387650', 0.4186962),
 ('100144571', 0.442406446),
 ('10014481', 0.4570644),
 ('1001528088', 0.4378227),
 ('100161002', 0.4317576),
 ('100172483', 0.447235167),
 ('100184113', 0.423039317),
 

In [60]:
node_dict = {}
citation_dict = {}
author_dict = {}
kw_dic = {}
title_dic = {}
# useful_keys = ['id','title','references','authors']
with open('data/subgraph_v11_1.json','r') as f:
    jsons = f.readlines()
for js in tqdm(jsons):
    js_l = json.loads(js)
    # js_l = dict((k,v) for k,v in js_l.items() if k in useful_keys)
    node_dict[js_l['id']] = js_l
    for aut in js_l['authors']:
        if aut['name'] not in author_dict:
            author_dict[aut['name']] = []
        author_dict[aut['name']].append(js_l['id'])
    for ref in js_l['references']:
        if ref not in citation_dict:
            citation_dict[ref] = []
        citation_dict[ref].append(js_l['id'])
    ind = js_l['id']
    title = js_l['title']
    title_dic[title] = ind
    if 'fos' not in js_l:
        continue
    for kw in js_l['fos']:
        kw_name = kw['name'].lower()
        if kw_name not in kw_dic:
            kw_dic[kw_name] = [(ind,kw['w'])]
        else:
            kw_dic[kw_name].append((ind,kw['w']))

100%|██████████| 614214/614214 [02:15<00:00, 4523.56it/s]


In [55]:
nds = {}
for idx in tqdm(node_dict):
    nds[idx] = get_message(idx)
    if 'references' in node_dict[idx]:
        nds[idx]['references'] = node_dict[idx]['references']
    else:
        nds[idx]['references'] = []

100%|██████████| 614214/614214 [00:21<00:00, 28189.52it/s]


In [50]:
sys.getsizeof(node_dict)/1e6

20.971616

In [63]:
ttl_dict = {}
for title in tqdm(title_dic):
    ttl_dict[title.lower()] = title_dic[title]

100%|██████████| 613323/613323 [00:00<00:00, 728420.56it/s]


In [64]:
title_dic = ttl_dict

In [49]:
import sys
sys.getsizeof(nds)/1e6

20.971616

In [52]:
import pickle

In [65]:
'''
citation_dict = {}
author_dict = {}
kw_dic = {}
title_dic = {}
'''
with open('data/processed_data.pkl','wb') as f:
    pickle.dump((nds,citation_dict,author_dict,kw_dic,title_dic),f)

In [11]:
import pickle
with open('data/processed_data.pkl','rb') as f:
    node_dict,citation_dict,author_dict,kw_dic,title_dic = pickle.load(f)

In [63]:
len(jsons)

614214

In [2]:
import json
from tqdm import tqdm

In [65]:
jsons[1]

'{"id": "1000018889", "title": "Remote Policy Enforcement for Trusted Application Execution in Mobile Environments", "authors": [{"name": "Fabio Martinelli", "id": "2107438709", "org": "Istituto di Informatica e Telematica, Consiglio Nazionale delle Ricerche, Pisa, Italy#TAB#"}, {"name": "Ilaria Matteucci", "id": "2311169168", "org": "Istituto di Informatica e Telematica, Consiglio Nazionale delle Ricerche, Pisa, Italy#TAB#"}, {"name": "Andrea Saracino", "id": "2140477378", "org": "Dipartimento di Ingegneria dell\'Informazione, Universit di Pisa, Pisa, Italy and Istituto di Informatica e Telematica, Consiglio Nazionale delle Ricerche, Pisa, Italy#TAB#"}, {"name": "Daniele Sgandurra", "id": "252575161", "org": "Istituto di Informatica e Telematica, Consiglio Nazionale delle Ricerche, Pisa, Italy#TAB#"}], "venue": {"raw": "international conference on trusted systems", "id": "2760518661"}, "year": 2013, "n_citation": 2, "page_start": "70", "page_end": "84", "doc_type": "Conference", "publ

In [26]:
def get_message(idx:str):
    node = node_dict[idx]
    paper_dict = {}
    paper_dict['title'] = node['title']
    paper_dict['authors'] = [f['name'] for f in node['authors']]
    if 'venue' in node:
        paper_dict['venue'] = node['venue']['raw']
    else:
        paper_dict['venue'] = ''
    paper_dict['year'] = node['year']
    paper_dict['citation'] = node['n_citation']
    if 'fos' in node:
        paper_dict['keywords'] = [f['name'] for f in sorted(node['fos'],key = lambda x:x['w'],reverse=True)][:5]
    else:
        paper_dict['keywords'] = []
    if "indexed_abstract" not in node:
        paper_dict['abstract'] = ""
    else:
        abst = ['' for _ in range(node['indexed_abstract']["IndexLength"])]
        for word,idxlist in node['indexed_abstract']["InvertedIndex"].items():
            for i in idxlist:
                abst[i] = word
        paper_dict['abstract'] = ' '.join(abst)
    return paper_dict

In [ ]:
kw_dic = {}
id_dic = {}
title_dic = {}
for js in tqdm(jsons):
    js_l = json.loads(js)
    ind = js_l['id']
    id_dic[ind] = js_l
    title = js_l['title']
    title_dic[title] = ind
    if 'fos' not in js_l:
        continue
    for kw in js_l['fos']:
        if kw['name'] not in kw_dic:
            kw_dic[kw['name']] = [(ind,kw['w'])]
        else:
            kw_dic[kw['name']].append((ind,kw['w']))

In [ ]:
len(kw_dic)

In [27]:
err = 0
for nd in tqdm(node_dict):
    get_message(nd)
print(err)

100%|██████████| 614214/614214 [00:19<00:00, 31318.16it/s]

0


In [17]:
print(get_message('1000018889')['abstract'])

Both in the cloud and mobile environments, a large number of online services is daily accessed through smartphones and tablets. Since several security, safety and trust concerns may arise when using these services, providers may require a usage policy to be enforced on the devices while accessing these services. This kind of policy enforcements enables service providers to have assurance that remote devices are in an acceptable state when using the provided service, according to their terms and conditions.

In this paper, we propose a framework which allows service providers to have assurance about the enforcement of some functional policies directly on the device. The proposed framework inserts an enforcer into the client's device, which is responsible for enforcing the provider's policy to abide by the terms and conditions of the service. To assure the integrity of the enforcer and of the policy, the framework exploits Trusted Computing techniques to remotely attest the enforcer's me

In [44]:
search_paper("Service provider")

[('1603584640', 0.8173289),
 ('166258374', 0.7909352),
 ('173477526', 0.7892417),
 ('114707311', 0.777953),
 ('1570821776', 0.7763443),
 ('168040670', 0.7695649),
 ('138700852', 0.7631655),
 ('1534787791', 0.7621108),
 ('1601837939', 0.7620162),
 ('1581296692', 0.7590559),
 ('148715279', 0.7545777),
 ('1505130406', 0.75436604),
 ('127904234', 0.753475368),
 ('1734589860', 0.7531088),
 ('1582035852', 0.7507168),
 ('16726377', 0.750234544),
 ('110519124', 0.750059068),
 ('117069272', 0.747506738),
 ('1509661363', 0.7463122),
 ('145822109', 0.7461096),
 ('160257375', 0.7450207),
 ('18718310', 0.744573),
 ('1561385173', 0.744305253),
 ('181242656', 0.743079364),
 ('1759074585', 0.74079597),
 ('141104829', 0.7400516),
 ('125132138', 0.7393041),
 ('183864699', 0.73688513),
 ('165489037', 0.7357138),
 ('1860902657', 0.7343438),
 ('1491496976', 0.7324376),
 ('1586145141', 0.7315028),
 ('1188031628', 0.7277101),
 ('1608051544', 0.7274931),
 ('1534167252', 0.7268608),
 ('105117948', 0.7253312),


In [38]:
kw_dic['Network']

KeyError: 'Network'

In [39]:
def search_paper(sentence:str):
    sentence = sentence.strip()
    ans_dic = {}
    if sentence in title_dic:
        ans_dic[title_dic[sentence]] = 10.0
    sentence = sentence.split(' ')
    for st in range(len(sentence)):
        for ed in range(st+1,len(sentence)+1):
            kw = ' '.join(sentence[st:ed])
            if kw in kw_dic:
                for idx,weight in kw_dic[kw]:
                    if idx not in ans_dic:
                        ans_dic[idx] = 0
                    ans_dic[idx] += weight
    return sorted(ans_dic.items(),key = lambda x:x[1],reverse = True)

In [22]:
len(citation_dict)

213661

In [19]:
a = [1,2,3,4,7]
for i in tqdm(a):
    print(i)

100%|██████████| 5/5 [00:00<00:00, 158.45it/s]

1
2
3
4
7


In [18]:
from tqdm import tqdm

In [23]:
import pickle

In [24]:
node_dict['1000018889']

{'id': '1000018889',
 'title': 'Remote Policy Enforcement for Trusted Application Execution in Mobile Environments',
 'authors': [{'name': 'Fabio Martinelli',
   'id': '2107438709',
   'org': 'Istituto di Informatica e Telematica, Consiglio Nazionale delle Ricerche, Pisa, Italy#TAB#'},
  {'name': 'Ilaria Matteucci',
   'id': '2311169168',
   'org': 'Istituto di Informatica e Telematica, Consiglio Nazionale delle Ricerche, Pisa, Italy#TAB#'},
  {'name': 'Andrea Saracino',
   'id': '2140477378',
   'org': "Dipartimento di Ingegneria dell'Informazione, Universit di Pisa, Pisa, Italy and Istituto di Informatica e Telematica, Consiglio Nazionale delle Ricerche, Pisa, Italy#TAB#"},
  {'name': 'Daniele Sgandurra',
   'id': '252575161',
   'org': 'Istituto di Informatica e Telematica, Consiglio Nazionale delle Ricerche, Pisa, Italy#TAB#'}],
 'references': ['1504669610', '1542792105', '1639158619']}

In [38]:
author_dict['2107438709']

['1000018889',
 '1000328014',
 '101667962',
 '113540453',
 '1172671697',
 '121425029',
 '135494744',
 '138746455',
 '139529891',
 '140180654',
 '1480908949',
 '1485377379',
 '1485725695',
 '1486199168',
 '1486853642',
 '1492629097',
 '1492794793',
 '1494423854',
 '1497836530',
 '1510961363',
 '1512963268',
 '1513989276',
 '1533718058',
 '1539700910',
 '1539721902',
 '1540359676',
 '1543782959',
 '1557188230',
 '1562425714',
 '1562827700',
 '1565275587',
 '1571942358',
 '1579534168',
 '1582567049',
 '1584984507',
 '1591991997',
 '1592544607',
 '1597048264',
 '1600323250',
 '1601816691',
 '1617766523',
 '1639158619',
 '1657886273',
 '1658613573',
 '1753645083',
 '175930347',
 '1773459340',
 '1825665527',
 '183206270']

In [43]:
a = 'AAA'
a.lower()

'aaa'

In [39]:
with open('data/processed_data_v11_1.pkl','wb') as f:
    pickle.dump((node_dict,citation_dict,author_dict),f)

In [ ]:
citation_dict['']

In [35]:
def gen_graph(index):
    index = str(index)
    g = nx.DiGraph()
    root_name = node_dict[index]['title']
    g.add_node(index,label = root_name,value = 1.0,modularity_class = 'self')
    for nd in node_dict[index]['references']:
        if len(g.nodes)<5:
            g.add_node(nd,label = node_dict[nd]['title'],value = get_value(index,nd), modularity_class = 'references')
            g.add_edge(nd,index)
    for nd in citation_dict.get(index,[]):
        if len(g.nodes)<10:
            g.add_node(nd,label = node_dict[nd]['title'],value = get_value(index,nd), modularity_class = 'cited')
            g.add_edge(index,nd)
    for aut in node_dict[index]['authors']:
        for nd in author_dict[aut['id']]:
            if nd not in g.nodes and len(g.nodes)<15:
                g.add_node(nd,label = node_dict[nd]['title'],value = get_value(index,nd), modularity_class = 'co-author')
                g.add_edge(index,nd)
    return g

In [36]:
g = gen_graph('1000018889')

In [ ]:
import time

In [29]:
import networkx as nx

In [31]:
def get_value(x,y):
    return 1.0

In [43]:
tic = time()
with open('data/processed_data_v11_1.pkl','rb') as f:
    node_dict,citation_dict,author_dict = pickle.load(f)
toc = time()
print(toc-tic)

11.968333959579468


In [42]:
from time import time
time()

1584241841.9177232

In [44]:
g = nx.DiGraph()
g.add_node('a',attr = 1)
g.add_node('b',attr = 2)
g.add_edge('a','b',val = 1)

In [47]:
list(dict(g.nodes).items())

[('a', {'attr': 1}), ('b', {'attr': 2})]

In [49]:
g.edges[('a','b')]

{'val': 1}

In [60]:
a = [3,2,1,0,1,2,3,5,0,1,2]
a.sort(reverse = True)
a

[5, 3, 3, 2, 2, 2, 1, 1, 1, 0, 0]

In [61]:
a = filter(lambda x:x>1,a)

In [62]:
list(a)

[5, 3, 3, 2, 2, 2]

In [18]:
import numpy as np

In [19]:
np.ceil(10.5)

11.0

In [3]:
import json

In [4]:
weight_dict = None
id2idx = None

def load_baseline():
    with open('data/edge_query_dict.json', 'r') as f:
        edge_query_dict = json.load(f)
    with open('data/id2idx.json', 'r') as f:
        id2idx = json.load(f)
    return edge_query_dict, id2idx

def query_weight(id_i, id_j, weight: dict, id2idx: dict) -> list:
    idx_i, idx_j = id2idx[id_i], id2idx[id_j]
    key = str(idx_i) + '_' + str(idx_j)
    queried_results = weight[key]
    return queried_results

In [5]:
weight_dict,id2idx = load_baseline()

In [73]:
weight_dict['1835_500005']

[0, 7484, 0.3333333333333333]

In [117]:
err = 0
for idx in tqdm(nds):
    nd = nds[idx]
    for ref in nd['references']:
        try:
            query_weight(idx,ref,weight_dict,id2idx)
        except:
            try:
                query_weight(ref,idx,weight_dict,id2idx)
            except:
                err += 1
    for aut in nd['authors']:
        for ref in author_dict[aut]:
            try:
                query_weight(idx,ref,weight_dict,id2idx)
            except:
                try:
                    query_weight(ref,idx,weight_dict,id2idx)
                except:
                    err += 1
print(err)

  0%|          | 175/614214 [00:31<37:23:52,  4.56it/s]

KeyboardInterrupt: 

In [118]:
err

3673

In [114]:
weight_dict['1835_1835']

[0.9802898074904512, 3, 0.7174531897448793]

In [8]:
idx2id = dict((v,k) for k,v in id2idx.items())

In [12]:
node_dict[idx2id[191760]]

{'title': 'The generalized MDL approach for summarization',
 'authors': ['Laks V. S. Lakshmanan',
  'Raymond T. Ng',
  'Christine Xing Wang',
  'Xiaodong Zhou',
  'Theodore Johnson'],
 'venue': 'very large data bases',
 'year': 2002,
 'citation': 47,
 'keywords': ['Automatic summarization',
  'Online analytical processing',
  'Data cube',
  'Minimum description length',
  'Data mining'],
 'abstract': 'There are many applications in OLAP and data analysis where we identify regions of interest. For example, in OLAP, an analysis query involving aggregate sales performance of various products in different locations and seasons could help identify interesting cells, such as cells of a data cube having an aggregate sales higher than a threshold. While a normal answer to such a quiry merely returns all interesting cells, it may be far more informative to the user if the system return summaries or descriptions of regions formed from the identified cells. The minimum Description Length (MDL) pr

In [13]:
node_dict[idx2id[110451]]

{'title': 'Personalizing XML text search in PIMENT',
 'authors': ['Sihem Amer-Yahia',
  'Irini Fundulaki',
  'Prateek Jain',
  'Laks V. S. Lakshmanan'],
 'venue': 'very large data bases',
 'year': 2005,
 'citation': 4,
 'keywords': ['Query expansion',
  'Ranking (information retrieval)',
  'Web query classification',
  'Sargable',
  'Web search query'],
 'abstract': "A growing number of text-rich XML repositories are being made available. As a result, more efforts have been deployed to provide XML full-text search that combines querying structure with complex conditions on text ranging from simple keyword search to sophisticated proximity search composed with stemming and thesaurus. However, one of the key challenges in full-text search is to match users' expectations and determine the most relevant answers to a full-text query. In this context, we propose query personalization as a way to take user profiles into account in order to customize query answers based on individual users' ne

In [7]:
len(id2idx)

614214

In [76]:
len(weight_dict)

4950226

In [6]:
num = 0
for eg in weight_dict:
    u,v = tuple(eg.split('_'))
    if v + '_' + u not in weight_dict:
        num += 1
print(num)

0


In [104]:
g = nx.DiGraph()
g.add_node('a')
g.add_node('b')
g.add_node('c')
g.add_edge('a','b')
g.add_edge('b','a')
g.add_edge('c','b')
g.add_edge('b','c')
adj = nx.adjacency_matrix(g).astype(float)

In [80]:
import networkx as nx

In [106]:
adj

<3x3 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [98]:
def print_adj(ad):
    for i in range(ad.shape[0]):
        print(ad[i].indices)

In [110]:
g = nx.DiGraph()
g.add_node('a')
g.add_node('b')
g.add_node('c')
g.add_edge('a','c')
g.add_edge('a','c')
g.add_edge('c','a')
adj_2 = nx.adjacency_matrix(g).astype(float)

In [111]:
adj_2

<3x3 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [108]:
adj_3 = adj_2 + adj

In [113]:
print_adj(adj)
print_adj(adj_2)
print_adj(adj_3)

[1]
[0 2]
[1]
[2]
[]
[0]
[1 2]
[0 2]
[0 1]


In [14]:
import numpy as np

In [18]:
st = set()
max_r = 10000000
for i in range(max_r):
    st.add((i,max_r-i))
a = [_[0] for _ in st]
b = [_[1] for _ in st]
from tqdm import tqdm
err = 0
for i in tqdm(range(max_r)):
    if a[i] + b[i] != max_r:
        err += 1
print(err)

100%|██████████| 10000000/10000000 [00:07<00:00, 1284735.15it/s]

0


In [24]:
g_aut = set()
for aut in tqdm(author_dict):
    papers = [id2idx[_] for _ in author_dict[aut]]
    for i in range(len(papers)):
        for j in range(i + 1,len(papers)):
            g_aut.add((papers[i],papers[j]))
            g_aut.add((papers[j],papers[i]))

100%|██████████| 604712/604712 [00:16<00:00, 35852.61it/s] 


In [20]:
err = 0
for i,j in tqdm(g_aut):
    if (j,i) not in g_aut:
        err += 1

100%|██████████| 19233579/19233579 [00:18<00:00, 1033406.52it/s]


In [21]:
err

0

In [25]:
len(g_aut)

19225850

In [23]:
for aut in tqdm(author_dict):
    author_dict[aut] = list(set(author_dict[aut]))

100%|██████████| 604712/604712 [00:01<00:00, 583978.51it/s]
